In [1]:
from src.llm_plan.environments.static.two_agents_blocksworld import TwoAgentsBlocksWorld

env = TwoAgentsBlocksWorld("./environments/static/TwoAgentsBlocksWorld.json")
env.plan  # Simple plan to solve the Two Agents Vault environment

[['AgentVowels.pddl', 'AgentConsonants.pddl'], ['orchestrator.pddl']]

In [2]:
# Create the plan
from src.llm_plan.planner import Planner
from src.llm_plan.llm import GPT_4o

model = GPT_4o()
planner = Planner(env, model)


In [3]:
# Plan
responses = planner.plan()

----- AgentVowels->pddl -----

----- System Prompt -----
	You are an expert in PDDL (Planning Domain Definition Language). Provide a domain and problem file for blocks whose names start with vowels.

----- Prompt -----
	Your name is AgentVowels. You are in a Blocksworld environment with the following public information:
['The world consists of blocks that can be stacked on one another or on the table.', 'An agent can hold at most one block at a time.', 'A block can only be moved if it is clear (nothing stacked on it).']
You have the following private knowledge:
['I can only move blocks whose names start with a vowel (A, E, I, O, U).', 'I cannot pick up a block if I am already holding one.']
This is your goal:
Help rearrange the blocks into the target configuration by moving vowel-named blocks.
Initial state:
{'blocks': ['A', 'B', 'C', 'E', 'O'], 'positions': {'A': 'table', 'B': 'A', 'C': 'table', 'E': 'table', 'O': 'C'}, 'goal_positions': {'A': 'B', 'B': 'table', 'C': 'E', 'E': 'table'

In [4]:
# Extract the pddl problem and domain from the LLM response
from src.llm_plan.parser import PDDLParser

pddl_parser = PDDLParser()

final_plan = responses["pddl_orchestrator"]
domain, problem = pddl_parser.parse(final_plan, from_file=False)

In [5]:
# Obtain the plan with the solver
import subprocess

from src.llm_plan.config import SOLVER_BINARY, SOLVER_ARGS

with open("./tmp/problem.pddl", "w") as f:
    f.write(problem)

with open("./tmp/domain.pddl", "w") as f:
    f.write(domain)
    
# Launch the solver
command = [
    SOLVER_BINARY,
    *SOLVER_ARGS,
    "./tmp/sas_plan",
    "./tmp/domain.pddl",
    "./tmp/problem.pddl",
]

with open("./tmp/logs.txt", "w") as logfile:
    subprocess.run(command, stdout=logfile, stderr=subprocess.STDOUT)